In [1]:
post = '/home/jupyter/datasets/xview/train/images/guatemala-volcano_00000000_post_disaster.png'
pre = '/home/jupyter/datasets/xview/train/images/guatemala-volcano_00000000_pre_disaster.png'

In [2]:
from PIL import Image
import numpy as np
import torch

import torch
torch.set_num_threads(1)

In [3]:
('8bh5rurv', 0.8848695158958435) 
('cnqyhmsb', 0.8836771249771118)
('ri1dvvam', 0.8832937479019165)
('ilhz9vkq', 0.883250892162323)

('ilhz9vkq', 0.883250892162323)

In [148]:
from xv import io
import ttach as tta
from imantics import Polygons
from xv.tta import BoxClassifierTTA

class ModelManager:
    LOCALIZATION_IDS = ('8bh5rurv', 'cnqyhmsb', 'ri1dvvam', 'ilhz9vkq')
    DAMSEG_IDS = ('tyi7q17m','ngmxec6s', 'vrgbzaup', '443t9312', 'pxcuoulb')
    DAMOD_IDS = ('kx9yzcb9', 'y7n4yzya', 'c1lit0gf')
    
    
    def __init__(self, base_dir='../models', device='cpu', size=(1024,1024)):
        self.base_dir = base_dir
        self.device = device
        self.size = size
    
    @staticmethod
    def load_img(img_path, image_mean = (0.485, 0.456, 0.406), image_std = (0.229, 0.224, 0.225)):
        image = np.array(Image.open(img_path))
        image = image.astype(np.float32)
        image /= 255.
        image = (image-image_mean)/image_std
        image = image.transpose(2,0,1)
        return torch.Tensor(image[None])
    
    @torch.no_grad()
    def predict_seg(self, im, model_subdir, run_id):
        path = f'{self.base_dir}/{model_subdir}/{run_id}'
        conf = io.Config(f'{path}/config.yaml')
        model, _ = io.load_segmentation_model(conf, f'{path}/state_dict.pth')
        model = model.eval().to(self.device)
        model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')
        return model(im.to(self.device)).sigmoid().cpu().numpy()[0]
    
    def predict_seg_dam(self, im, localization, run_id, mean_ag=True):
        probs = self.predict_seg(im, 'dam_seg', run_id)
        probs = probs / probs.sum(0)
        if mean_ag:
            _probs = np.zeros(probs.shape)
            for poly in Polygons.from_mask(localization):
                poly_mask = Polygons.create([poly]).mask(*self.size).array
                poly_prob = probs[:,poly_mask].mean(1, keepdims=True)
                _probs[:,poly_mask] = poly_prob
                probs = _probs
        return probs
    
    @torch.no_grad()
    def predict_seg_od(self, im, localization, run_id):
        probs = np.zeros((4, *self.size))
        polys = Polygons.from_mask(localization)
        polypoints = polys.points
        
        if not polypoints:
            return probs
        
        path = f'{self.base_dir}/dam_od/{run_id}'
        conf = io.Config(f'{path}/config.yaml')
        model = io.load_damage_model(conf, f'{path}/state_dict.pth')
        model = BoxClassifierTTA(model)
        model = model.eval().cuda()

        boxes = torch.Tensor([[min(p[:,0]), min(p[:,1]), max(p[:,0]), max(p[:,1])] for p in polypoints])
        box_probs = model(im.to(self.device), [boxes.to(self.device)]).sigmoid().cpu().numpy()
        box_probs /= box_probs.sum(1, keepdims=True)
        
        for poly, box_prob in zip(polypoints, box_probs):
            poly_mask = Polygons([poly]).mask(*self.size).array
            probs[:, poly_mask] = box_prob[:,None]

        return probs
        
    
    def predict_localization(self, pre_im_path, post_im_path, run_ids, thresh=0.5):
        pre_im = self.load_img(pre_im_path)
        out = self.predict_seg(pre_im, 'loc', run_ids[0])
        for run_id in run_ids[1:]:
            out += self.predict_seg(pre_im, None, run_id)
        out /= len(run_ids)
        return (out > thresh).astype(np.uint8)[0]
    
    def predict_damage(self, post_im_path, localization, damseg_ids, damod_ids):
        post_im = self.load_img(post_im_path)
        localization = localization.astype(np.bool)
        probs = np.zeros((4, *self.size))
        
        for run_id in damseg_ids:
            pass

In [149]:
%pdb on

Automatic pdb calling has been turned ON


In [150]:
mm = ModelManager(device='cuda')

In [151]:
im = mm.load_img(post)

In [152]:
prob_out = mm.predict_seg_dam(im, localization, mm.DAMSEG_IDS[0])

<All keys matched successfully>


In [153]:
prob_out = mm.predict_seg_od(im, localization, mm.DAMOD_IDS[0])

<All keys matched successfully>


In [100]:
ar = np.zeros((4,10))

In [101]:
ar[:,[0,1,2]] = (1,2,3,5)

ValueError: shape mismatch: value array of shape (4,) could not be broadcast to indexing result of shape (3,4)

> <ipython-input-101-a56b17761017>(1)<module>()
----> 1 ar[:,[0,1,2]] = (1,2,3,5)



ipdb>  q


In [67]:
prob_out[:,0,0]

array([0.868465  , 0.4529124 , 0.26270437, 0.20301004], dtype=float32)

In [33]:
%time localization = mm.predict_localization(pre, post, mm.LOCALIZATION_IDS, 0.175)

<All keys matched successfully>


<All keys matched successfully>


<All keys matched successfully>


<All keys matched successfully>
CPU times: user 9.68 s, sys: 2.14 s, total: 11.8 s
Wall time: 12.8 s


In [22]:
run_id = '8bh5rurv'
path = f'{mm.base_dir}/loc/{run_id}'

In [ ]:
('kx9yzcb9', 0.8188149291818385),
('y7n4yzya', 0.808878301295482),
('c1lit0gf', 0.8086308781844662),

In [24]:
%time mod = mm.predict_seg(mm.load_img(pre), None, '8bh5rurv')

<All keys matched successfully>
CPU times: user 41.3 s, sys: 6.18 s, total: 47.4 s
Wall time: 19 s


In [28]:
mod.shape

(1, 1, 1024, 1024)

In [15]:
%time out = mm.predict_seg(mm.load_img(pre), None, '8bh5rurv')

<All keys matched successfully>
CPU times: user 27.9 s, sys: 6.62 s, total: 34.5 s
Wall time: 9.08 s


In [23]:
localization = localization.astype(np.uint8)

In [ ]:
# segment
('8bh5rurv', 0.8848695158958435) 
('cnqyhmsb', 0.8836771249771118)
('ri1dvvam', 0.8832937479019165)
('ilhz9vkq', 0.883250892162323)

In [32]:
def predict_proba_loc(pre, post, run_id):
    pass

def predict_proba_dam(pre, post, loc, run_id):
    pass

def predict_proba_dam_od(pre, post, loc, run_id):
    pass

def predict(pre, post):
    pass

In [9]:
%load_ext autoreload
%autoreload 2

import wandb

In [10]:
seg_runs = [
    ('8bh5rurv', 0.8848695158958435),
    ('cnqyhmsb', 0.8836771249771118),
    ('ri1dvvam', 0.8832937479019165),
    ('ilhz9vkq', 0.883250892162323),
    ('dhmhz041', 0.8829216957092285),
    ('7mrir9a3', 0.8828873634338379),
    ('yzmxfvrv', 0.8828799724578857),
    ('5ywrt6qn', 0.882686972618103),
    ('ighbwtdm', 0.8826081156730652),
    ('315hd0sm', 0.8825200796127319),
    ('y2jgqdox', 0.8821526169776917),
    ('xr44njvv', 0.8815780282020569),
    ('mb22v5ps', 0.8812705874443054),
    ('q6d1n08d', 0.8809899091720581),
    ('37y0hk18', 0.8808367848396301),
    ('3ag6fjep', 0.8806321024894714),
    ('uevn0e5g', 0.8803757429122925),
    ('0en8eomx', 0.8799382448196411),
    ('o0wao11j', 0.8796645998954773),
    ('4qrallkn', 0.8785540461540222),
    ('6vya7uxd', 0.8780744671821594),
    ('rwa0acur', 0.8775778412818909),
    ('qoijsx0h', 0.8775392770767212),
    ('l1hf5uuh', 0.8767494559288025),
    ('ebwyd2di', 0.8765263557434082),
    ('6g0nkv0j', 0.8760395646095276),
    ('ph4c64ar', 0.8760202527046204),
    ('amkk3cbw', 0.8735491037368774),
    ('c1dlbdub', 0.8734628558158875),
    ('hiinbqsu', 0.8719046115875244)
]

seg_dam_runs = [
    ('tyi7q17m', 0.8150793196256518),
    ('ngmxec6s', 0.8096530334019485),
    ('vrgbzaup', 0.8096383585083926),
    ('443t9312', 0.8083381310832123),
    ('pxcuoulb', 0.8081565006839244),
    ('cpumt46r', 0.8080391959243641),
    ('viptnxip', 0.8076960005628842),
    ('mhczmsoa', 0.8073159432491346),
    ('znxzcx0a', 0.8072744354969165),
    ('7ar6am07', 0.8071092834046755),
    ('lwe6n5c0', 0.8069356023059625),
    ('90mnu920', 0.8069068799555296),
    ('08meqrfc', 0.806507592996532),
    ('j4xmz3we', 0.8061300715307924),
    ('5ocdy37k', 0.806018066403344),
    ('5pn3bp2o', 0.8057287190805172),
    ('0gvvydkt', 0.8057094304994589),
    ('or34wjnl', 0.8056978455958853),
    ('8n17nhye', 0.8053004703929182),
    ('c2hvohg7', 0.8050853487741387),
    ('33aasxzr', 0.8048996430920557),
    ('40l1n8y6', 0.804133361539285),
    ('8slg69rb', 0.8039897395816178),
    ('yzy5xz0v', 0.8037504662087867),
    ('71urz3vc', 0.803504510967261),
    ('plef5k0s', 0.8033634254385499),
    ('rfu2ifuh', 0.8031063433726832),
    ('5zu1m3bn', 0.8028573049710269),
    ('nj4eqar6', 0.8022980617768509),
    ('95pmh37c', 0.8022721609563958)
]

dam_od_runs = [
    ('kx9yzcb9', 0.8188149291818385),
    ('y7n4yzya', 0.808878301295482),
    ('c1lit0gf', 0.8086308781844662),
    ('ygnsmpis', 0.8047019871807307),
    ('5v3elkfr', 0.8026197042480495),
    ('0qjy2h9e', 0.8011403695871695),
    ('6s629jji', 0.7983212991363049),
    ('umj3wmwh', 0.7972560052280561),
    ('phtv4ekc', 0.7952364835570059),
    ('ypydqpw5', 0.7950183345724261),
    ('9bv7h3cw', 0.7941133157892892),
    ('w8ka9fat', 0.7932566433728835),
    ('zu0uhkyk', 0.7921109978206695)
]

## Building Ensemble

In [11]:
import numpy as np

In [14]:
TOP_DAM_SEG = 5

seg_dam_ids = [r_id for r_id, _ in seg_dam_runs[:TOP_DAM_SEG]]

In [5]:
from tqdm.notebook import tqdm
y_d = np.load('../dev/damage.npy')
y_d += 1

y_d_pred = np.load(f'../dev/segment-damage/{seg_dam_ids[0]}.npy')
y_d_pred /= y_d_pred.sum(1, keepdims=True)

for run_id in tqdm(seg_dam_ids[1:]):
    probs = np.load(f'../dev/segment-damage/{run_id}.npy')
    probs /= probs.sum(1, keepdims=True)
    y_d_pred += probs
del probs
y_d_pred /= len(seg_dam_ids)
#y_d_pred = y_d_pred.argmax(1)
#y_d_pred += 1

In [40]:
msk_dmg = y_d_pred.argmax(1) + 1

In [13]:
TOP_DAM_OD = 4
dam_od_ids = [r_id for r_id, _ in dam_od_runs[:TOP_DAM_OD]]

In [7]:
y_d_preds = np.stack([
    np.load(f'../dev/y_pred/damage-od/{run_id}.npy')
] for run_id in tqdm(dam_od_ids))

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
top_n = 2

In [11]:
od_msk.sum(-1, keepdims=True).shape

(2, 280, 1024, 1024, 1)

In [18]:
top_n = 4
od_msk = y_d_preds[:top_n,0]
od_msk = (od_msk/od_msk.sum(-1, keepdims=True)).mean(0).transpose(0,3,1,2)

msk_dmg = ((len(seg_dam_ids)*y_d_pred + top_n*od_msk)).argmax(1) + 1

MemoryError: Unable to allocate array with shape (4, 280, 1024, 1024, 4) and data type float32

In [27]:
msk_dmg = y_d_preds[0,0].argmax(-1) + 1

In [ ]:
## def EVAL

from xv.submission_metrics import RowPairCalculator, F1Recorder

y_d_pred_filtered = msk_dmg * y_pred
y_pred_damage = y_d_pred_filtered[y]
y_true_damage = y_d[y]
damage_f1s = [F1Recorder(*RowPairCalculator.compute_tp_fn_fp(y_pred_damage, y_true_damage, cls), cls) for cls in range(1,5)]

building_f1 = F1Recorder(*RowPairCalculator.compute_tp_fn_fp(y_pred, y, 1), 'building')
xs = [d.f1 for d in damage_f1s]
damage_score = len(xs) / sum((x+1e-6)**-1 for x in xs)
building_score = building_f1.f1
score = 0.3*building_score + 0.7*damage_score

In [ ]:
building_score, damage_score, score

In [14]:
building_score, damage_score, score # 2 od 

(0.867134416729549, 0.8047028070200527, 0.8234322899329016)

In [19]:
building_score, damage_score, score

(0.867134416729549, 0.8025184281356607, 0.8219032247138272)

In [24]:
building_score, damage_score, score # 5 

(0.867134416729549, 0.8039647871455755, 0.8229156760207674)

In [30]:
building_score, damage_score, score # 6

(0.867134416729549, 0.8033511890136883, 0.8224861573284465)

In [42]:
building_score, damage_score, score # .9 decay

(0.867134416729549, 0.8038322889349666, 0.8228229272733414)

In [ ]:
building_score, damage_score, score

In [9]:
import numpy as np
y_pred = np.load('../dev/y_pred/y_pred.npy')

In [15]:
TOP_SEGMENT = 4
seg_ids = [r_id for r_id, _ in seg_runs[:TOP_SEGMENT]]

In [10]:
y = np.load('../dev/buildings.npy')

#X = [np.load(f'../dev/segment/{run_id}.npy') for run_id in seg_ids]
#X = np.stack(X, axis=0).mean(0)

y = y.astype(np.bool)
#X = X.astype(np.float16)

In [ ]:
X.shape, y.shape

In [61]:
thresh = 0.175
y_pred = (X[:,0] > thresh)

In [21]:
TOP_DAM_SEG = 5
TOP_DAM_OD = 5

seg_dam_ids = [r_id for r_id, _ in seg_dam_runs[:TOP_DAM_SEG]]
dam_od_ids = [r_id for r_id, _ in dam_od_runs[:TOP_DAM_OD]]

In [ ]:

#PROJECT = 'building-seg-damage'
#METRIC = "hmean:damage:categorical:f1"

#PROJECT = 'building-damage'
#METRIC = "hmean:damage:categorical:f1"



In [25]:
for rid in seg_dam_ids:
    project = 'building-seg-damage'
    path = f"xvr-hlt/{project}/{rid}"
    conf_file = wandb.restore('config.yaml', run_path=path, replace=True).name
    state_file = wandb.restore('state_dict.pth', run_path=path, replace=True).name
    pth = f'../models/dam_seg/{rid}/'
    !mkdir {pth}
    !mv config.yaml {pth}
    !mv state_dict.pth {pth}

In [26]:
for rid in dam_od_ids:
    project = 'building-damage'
    path = f"xvr-hlt/{project}/{rid}"
    conf_file = wandb.restore('config.yaml', run_path=path, replace=True).name
    state_file = wandb.restore('state_dict.pth', run_path=path, replace=True).name
    pth = f'../models/dam_od/{rid}/'
    !mkdir {pth}
    !mv config.yaml {pth}
    !mv state_dict.pth {pth}

In [30]:
seg_ids

['8bh5rurv', 'cnqyhmsb', 'ri1dvvam', 'ilhz9vkq']

In [31]:
for rid in seg_ids:
    project = 'sky-eye-full'
    path = f"xvr-hlt/{project}/{rid}"
    conf_file = wandb.restore('config.yaml', run_path=path, replace=True).name
    state_file = wandb.restore('state_dict.pth', run_path=path, replace=True).name
    pth = f'../models/loc/{rid}/'
    !mkdir {pth}
    !mv config.yaml {pth}
    !mv state_dict.pth {pth}

In [20]:
conf_file, state_file

('./config.yaml', './state_dict.pth')

In [22]:
!ls ../models/

dam_od	dam_seg  loc


In [ ]:
seg_run_path = f"xvr-hlt/{project}/{run_id}"
conf_file = wandb.restore('config.yaml', run_path=seg_run_path, replace=True).name
state_file = wandb.restore('state_dict.pth', run_path=seg_run_path, replace=True).name

In [11]:
from xv import io
import ttach as tta
import torch
from tqdm import tqdm_notebook as tqdm
import numpy as np
from xv.tta import BoxClassifierTTA
from imantics import Polygons

@torch.no_grad()
def predict_seg_run(run_id, building_preds, project, to):
    seg_run_path = f"xvr-hlt/{project}/{run_id}"
    conf_file = wandb.restore('config.yaml', run_path=seg_run_path, replace=True).name
    state_file = wandb.restore('state_dict.pth', run_path=seg_run_path, replace=True).name
    conf = io.Config(conf_file)
    conf.filter_none = False # predict on all
    
    model, preproc = io.load_segmentation_model(conf, state_file)
    model.eval().cuda();
    model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')
    
    dev_data, dev_loader = io.load_dev_data(conf, preproc)
    
    ars = []
    
    for (im, _), y_pred in tqdm(zip(dev_loader, building_preds)):
        out = model(im.cuda()).sigmoid()
        
        damage_map = np.zeros((1024,1024))
        for poly in Polygons.from_mask(y_pred):
            poly_mask = Polygons.create([poly]).mask(1024, 1024).array
            cls = out[:,poly_mask].mean(1).argmax() + 1
            damage_map[poly_mask] = cls
        ars.append(out.cpu().numpy())
    
    preds = np.concatenate(ars)
    np.save(f'{to}/{run_id}', preds)


from imantics import Polygons
Polygons.create([poly]).mask(1024, 1024).array
    
@torch.no_grad()
def predict_od(run_id, project, to):
    seg_run_path = f"xvr-hlt/{project}/{run_id}"
    conf_file = wandb.restore('config.yaml', run_path=seg_run_path, replace=True).name
    state_file = wandb.restore('state_dict.pth', run_path=seg_run_path, replace=True).name
    conf = io.Config(conf_file)

    dmg_model = io.load_damage_model(conf, state_file)
    dmg_model = BoxClassifierTTA(dmg_model)
    dmg_model = dmg_model.eval().cuda()
    
    for (im, _), y_pred in tqdm(zip(dev_loader, building_preds)):
        pass
    return dmg_model

In [ ]:
from xv import io


import torch
from tqdm import tqdm_notebook as tqdm
import numpy as np
from xv.tta import BoxClassifierTTA

@torch.no_grad()
def predict_seg_run(run_id, project, to):
    seg_run_path = f"xvr-hlt/{project}/{run_id}"
    conf_file = wandb.restore('config.yaml', run_path=seg_run_path, replace=True).name
    state_file = wandb.restore('state_dict.pth', run_path=seg_run_path, replace=True).name
    conf = io.Config(conf_file)
    conf.filter_none = False # predict on all
    
    model, preproc = io.load_segmentation_model(conf, state_file)
    model.eval().cuda();
    model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')
    
    dev_data, dev_loader = io.load_dev_data(conf, preproc)
    
    ars = []
    
    for im, mask in tqdm(dev_loader):
        out = model(im.cuda()).sigmoid()
        ars.append(out.cpu().numpy())
    
    preds = np.concatenate(ars)
    np.save(f'{to}/{run_id}', preds)

@torch.no_grad()
def predict_od(run_id, project, to):
    return dmg_model

In [44]:
import segmentation_models_pytorch as smp
from xv import io
import numpy as np

conf=io.Config('config.yaml')

conf.n_cpus = 8
conf.eval_resolution = 1024
conf.mode = 'categorical'
conf.filter_none = False
conf.dual_input = False
preproc_fun = smp.encoders.get_preprocessing_fn('efficientnet-b7')
dev_data, dev_loader = io.load_dev_data(conf, preproc_fun)
damage_ims = np.stack([im for im, _ in tqdm(dev_data)])

/home/jupyter/sky-eye/xv/io.py:37: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  _conf = yaml.load(file_path)
100%|██████████| 280/280 [00:04<00:00, 58.20it/s]


In [49]:
from xv import io
from tqdm.notebook import tqdm
from xv.tta import BoxClassifierTTA
import torch
from tqdm.notebook import tqdm
from imantics import Polygons
import cv2


run_id = dam_od_ids[0]
project = 'building-damage'

@torch.no_grad()
def get_msk_dmgs(run_id, project, damage_ims, y_pred):
    seg_run_path = f"xvr-hlt/{project}/{run_id}"
    conf_file = wandb.restore('config.yaml', run_path=seg_run_path, replace=True).name
    state_file = wandb.restore('state_dict.pth', run_path=seg_run_path, replace=True).name
    conf = io.Config(conf_file)
    dmg_model = io.load_damage_model(conf, state_file)

    dmg_model = BoxClassifierTTA(dmg_model)
    dmg_model = dmg_model.eval().cuda()
    
    msk_dmgs = np.zeros((280, 1024, 1024, 4), dtype=np.float32)
    for msk_dmg, damage_im, building_pred in tqdm(zip(msk_dmgs, damage_ims, y_pred)):
        polys = Polygons.from_mask(building_pred)
        polypoints = polys.points

        if polypoints:
            boxes = torch.Tensor([[min(p[:,0]), min(p[:,1]), max(p[:,0]), max(p[:,1])] for p in polypoints])
            out = dmg_model(torch.Tensor(damage_im)[None].cuda(), [boxes.cuda()])
            #assert False
            probs = out.sigmoid().cpu().numpy()
            for poly, prob in zip(polypoints, probs):
                msk_dmg[Polygons([poly]).mask(1024,1024).array] = prob
    return msk_dmgs

In [ ]:
from xv.submission_metrics import RowPairCalculator, F1Recorder

y_d_pred_filtered = msk_dmg * y_pred
y_pred_damage = y_d_pred_filtered[y]
y_true_damage = y_d[y]
damage_f1s = [F1Recorder(*RowPairCalculator.compute_tp_fn_fp(y_pred_damage, y_true_damage, cls), cls) for cls in range(1,5)]

building_f1 = F1Recorder(*RowPairCalculator.compute_tp_fn_fp(y_pred, y, 1), 'building')
xs = [d.f1 for d in damage_f1s]
damage_score = len(xs) / sum((x+1e-6)**-1 for x in xs)
building_score = building_f1.f1
score = 0.3*building_score + 0.7*damage_score

In [ ]:
building_score, damage_score, score

In [28]:
building_score, damage_score, score

(0.867134416729549, 0.7787355957896358, 0.8052552420716097)

In [29]:
building_score, damage_score, score

(0.867134416729549, 0.7787355957896358, 0.8052552420716097)

In [17]:
building_score, damage_score, score

(0.867134416729549, 0.7835007926782325, 0.8085908798936274)

In [64]:
building_score, score, damage_score

(0.867134416729549, 0.8085908798936274, 0.7835007926782325)

In [21]:
import torch
from imantics import Polygons
import cv2

In [ ]:
for run_id in tqdm(seg_ids[10:]):
    predict_seg_run(run_id, 'sky-eye-full', '../dev/segment/')

In [ ]:
for run_id in tqdm(seg_dam_ids[:10]):
    predict_seg_run(run_id, 'building-seg-damage', '../dev/segment-damage/')

In [ ]:
import wandb

api = wandb.Api()

PROJECT = 'sky-eye-full'
METRIC = "building:f1"

#PROJECT = 'building-seg-damage'
#METRIC = "hmean:damage:categorical:f1"

#PROJECT = 'building-damage'
#METRIC = "hmean:damage:categorical:f1"


runs = api.runs(f'xvr-hlt/{PROJECT}')

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm_notebook as tqdm

with ThreadPoolExecutor(16) as pool:
    scores = list(tqdm(pool.map(lambda r: get_score(r, metric=METRIC), runs)))
    
scores = dict(zip(runs, scores))

kvs = []
vs = set()

for k,v in sorted(((k,v) for k,v in scores.items() if v is not None), key=lambda kv: -kv[1]):
    if v in vs:
        continue
    vs.add(v)
    kvs.append((k,v))

In [ ]:
### Def get Poly-average

from imantics import Polygons
from xv.submission_metrics import RowPairCalculator, F1Recorder

ars = np.zeros((280,1024,1024), dtype=np.uint8)

for yb_pred, out, damage_map in tqdm(zip(y_pred, y_d_pred, ars)):
    for poly in Polygons.from_mask(yb_pred):
        poly_mask = Polygons.create([poly]).mask(1024, 1024).array
        cls = out[:,poly_mask].mean(1).argmax() + 1
        damage_map[poly_mask] = cls
        

y_d_pred_filtered = ars * y_pred
y_pred_damage = y_d_pred_filtered[y]
y_true_damage = y_d[y]
damage_f1s = [F1Recorder(*RowPairCalculator.compute_tp_fn_fp(y_pred_damage, y_true_damage, cls), cls) for cls in range(1,5)]

building_f1 = F1Recorder(*RowPairCalculator.compute_tp_fn_fp(y_pred, y, 1), 'building')
xs = [d.f1 for d in damage_f1s]
damage_score = len(xs) / sum((x+1e-6)**-1 for x in xs)
building_score = building_f1.f1
score = 0.3*building_score + 0.7*damage_score

In [ ]:
### Find threshes

threshes = np.arange(0.15, 0.25, 0.0125)

from xv.submission_metrics import RowPairCalculator, F1Recorder

from pprint import pprint
from IPython.display import Markdown, display

for thresh in threshes:
    print(f"### {thresh: .3f}")
    y_pred = (X[:,0] > thresh)
    y_d_pred_filtered = y_d_pred * y_pred
    y_pred_damage = y_d_pred_filtered[y]
    y_true_damage = y_d[y]
    damage_f1s = [F1Recorder(*RowPairCalculator.compute_tp_fn_fp(y_pred_damage, y_true_damage, cls), cls) for cls in range(1,5)]
    building_f1 = F1Recorder(*RowPairCalculator.compute_tp_fn_fp(y_pred, y, 1), 'building')
    xs = [d.f1 for d in damage_f1s]
    damage_score = len(xs) / sum((x+1e-6)**-1 for x in xs)
    building_score = building_f1.f1
    score = 0.3*building_score + 0.7*damage_score
    print(f"building: {building_score: .4f}, 'damage': {damage_score: .4f}, overall: {score: .6f}\n")